In [3]:
import pandas as pd
import numpy as np
import os, sys

In [4]:
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

C:\Users\mavinaya\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
# expected image size
img_width, img_height = 150, 150
# folder containing the images on which
# the network will train. The train folder 
# has two sub folders, dogs and cats.
location = 'D:/Users/Default User/Desktop/ML/car-damage-dataset'

train_data_dir = location+'/data1/training'
# folder containing the validation samples
# folder structure is same as the training folder
validation_data_dir = location+'/data1/validation'
# how many images to be considered for training
train_samples = 10
# how many images to be used for validation
validation_samples = 4
# how many runs will the network make
# over the training set before starting on
# validation
epoch = 50

# path to the model weights files.
weights_path = 'D:\\Users\\Default User\\Desktop\\ML\\car-damage-dataset\\vgg16_weights.h5'
top_model_weights_path = 'D:\\Users\\Default User\\Desktop\\ML\\car-damage-dataset\\fc_model.h5'

In [10]:
#Transfer learning with VGG16 
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [17]:
## set model architechture 
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [18]:
batch_size = 32 # tune it
epochs = 5 # increase it lb:0.40856(50 epochs), 0.27101 (5 epochs)

'''
train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)
train_datagen.fit(x_train)
'''

# All images will be rescaled by 1./255
datagen = ImageDataGenerator(rescale=1./255)
      
train_generator = datagen.flow_from_directory(train_data_dir,
                                            target_size=(img_width, img_height),
                                            batch_size=16, 
                                            class_mode='binary', 
                                            shuffle=False) 
validation_generator = datagen.flow_from_directory(validation_data_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 10 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch=100, epochs=15, validation_data=validation_generator,
      validation_steps=50)

Epoch 1/15
100/100 [==============================] - 720s 7s/step - loss: 0.1390 - acc: 0.9630 - val_loss: 0.3339 - val_acc: 1.0000
Epoch 2/15
100/100 [==============================] - 828s 8s/step - loss: 0.0049 - acc: 1.0000 - val_loss: 0.2985 - val_acc: 1.0000
Epoch 3/15
100/100 [==============================] - 1270s 13s/step - loss: 0.0022 - acc: 1.0000 - val_loss: 0.2840 - val_acc: 1.0000
Epoch 4/15
100/100 [==============================] - 1247s 12s/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.2745 - val_acc: 1.0000
Epoch 5/15
100/100 [==============================] - 1212s 12s/step - loss: 9.2043e-04 - acc: 1.0000 - val_loss: 0.2674 - val_acc: 1.0000
Epoch 6/15
100/100 [==============================] - 1311s 13s/step - loss: 6.8743e-04 - acc: 1.0000 - val_loss: 0.2621 - val_acc: 1.0000
Epoch 7/15
100/100 [==============================] - 1299s 13s/step - loss: 5.4009e-04 - acc: 1.0000 - val_loss: 0.2572 - val_acc: 1.0000
Epoch 8/15
100/100 [===========================